In [ ]:
import os, glob, re
from utils.readYaml import readConfigYaml
from utils.format_tousands import format_tousands
from tqdm import tqdm
import numpy as np
import pandas as pd
import pdb
import scipy.stats as stats

# from IPython.display import ,HTML
from IPython.core.display import display,HTML
import subprocess

# format for float to suppress/enable scientific notation
# pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.float_format', '{:.2g}'.format)

import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnchoredText
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [ ]:
import seaborn as sns
sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (20.0, 10.0)
plt.rcParams['savefig.dpi'] = 90
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.size'] = 18
plt.rcParams['agg.path.chunksize'] = 10000

In [ ]:
#comment this part if you want to run the notebook through the python script
# os.environ['FOLDER'] = "outputs/FullTest20200526/OutSample_PER/1M/('KLM', '[50000, 10000, 200000]')_('max_experiences', '250000')_('use_PER', 'False')_('final_lr', '0.0001')"
# os.environ['N_TRAIN'] = '1M'
# os.environ['FOLDER'] = "outputs/FullTest20200528/OutSample_RLparams/1M/('steps_to_min_eps', '500000')_('max_experiences', '500000')_('copy_step', '5000')_('gamma', '0.9')"
# os.environ['N_TRAIN'] = '1M'


In [ ]:
# copy data directory and filename for results then import config
data_dir = os.environ['FOLDER']
resfilename = 'Results_{}.parquet'.format(os.environ['N_TRAIN'])
tablefilename = 'QTable{}.parquet'.format(os.environ['N_TRAIN'])
p = readConfigYaml(os.path.join(data_dir,'config_{}.yaml'.format(os.environ['N_TRAIN'])))
p_post = readConfigYaml('ExPostresults_Config.yaml')
if p['out_of_sample_test']:
    testresfilename = 'TestResults_{}.parquet'.format(format_tousands(p['N_test']))

# Experiment parameters

In [ ]:
print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
print('MODEL')
print()
print(os.path.split(data_dir)[-1])
print()
print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')

In [ ]:
for k, v in p.items():
    print (k,':',v)

In [ ]:
for k, v in p_post.items():
    print (k,':',v)

In [ ]:
df = pd.read_parquet(os.path.join(data_dir,resfilename+'.gzip'))
if p['executeRL']: 
    Q = pd.read_parquet(os.path.join(os.getcwd(),data_dir,tablefilename+'.gzip'))
if p['out_of_sample_test']:
    df_test = pd.read_parquet(os.path.join(data_dir,testresfilename+'.gzip'))

# Epsilondecay, learning rate decay and betaPER growth

Visualize the rate of decay for the parameter $\epsilon$ for $\epsilon$-greedy action selection of DQN algorithm and the rate of decay for the initial learning rate

In [ ]:
def eps(N,epsilon,eps_decay,min_eps):
    
    eps = []
    
    for i in range(N):
        epsilon = max(min_eps, epsilon - eps_decay)
        eps.append(epsilon)
        
    return np.array(eps)


def lr_exp_decay(N,initial_learning_rate,decay_rate,decay_steps):
    
    lrs = []
    
    for i in range(N):
        lr = initial_learning_rate * decay_rate ** (i / decay_steps)
        lrs.append(lr)
    
    return np.array(lrs)

def beta_anneal(N,beta0,beta_growth,finalbeta):
    betas = []
    
    for i in range(N):
        beta0 = min(finalbeta, beta0 + beta_growth)
        betas.append(beta0)
        
    return np.array(betas)

In [ ]:
N = p['N_train']
epsilon = p['epsilon']
eps_decay = p['eps_decay']
min_eps = p['min_eps']
e = eps(N,epsilon,eps_decay,min_eps)

if p['lr_schedule'] == 'exponential':
    initial_learning_rate = p['learning_rate']
    decay_rate = p['exp_decay_rate']
    decay_steps = p['exp_decay_steps']
    l = lr_exp_decay(N,initial_learning_rate,decay_rate,decay_steps)

if p['use_PER']:
    beta0 = p['PER_b'] 
    beta_growth = p['PER_b_growth']
    finalbeta = p['final_PER_b'] 
    b =  beta_anneal(N,beta0,beta_growth,finalbeta)
    
if p['use_PER']:
    a0 = p['PER_a'] 
    a_growth = p['PER_a_growth']
    finala = p['final_PER_a'] 
    b =  beta_anneal(N,a0,a_growth,finala)
    
plt.rcParams['font.size'] = 17
fig = plt.figure(figsize=(40,30))

ax1 = fig.add_subplot(4,1,1)
ax1.plot(e)
ax1.set_title('Epsilon greedy decay over iterations')
ax1.legend(['epsilon_greedy'])
ax1.set_ylabel('Value')
ax1.set_xlabel('Iteration')
if p['lr_schedule'] == 'exponential':
    ax2 = fig.add_subplot(4,1,2)
    ax2.plot(l)
    ax2.set_title('Exponential learning rate decay over iterations')
    ax2.legend([ 'learning_rate'])
    ax2.set_xlabel('Iteration')
if p['use_PER']:
    ax3 = fig.add_subplot(4,1,3)
    ax3.plot(b)
    ax3.set_title('Beta PER growth over iterations')
    ax3.legend([ 'Beta_PER'])
    ax3.set_xlabel('Iteration')
    
    ax4 = fig.add_subplot(4,1,4)
    ax4.plot(b)
    ax4.set_title('Alpha PER growth over iterations')
    ax4.legend([ 'Alpha_PER'])
    ax4.set_xlabel('Iteration')

# Stats

In [ ]:
df.head()

DQN stats

In [ ]:
# stats for algorithm solution
if p['executeDRL']: 
    display(df.filter(like='_DQN').describe())

TabQ stats (if any)

In [ ]:
# stats for optimal solution
if p['executeRL']:
    display(df.filter(like='_Q').describe())

Optimal stats

In [ ]:
if p['executeGP']:
    display(df.filter(like='Opt').describe())

Comparison with optimal stats

In [ ]:
# compare the two
if p['executeDRL']: 
    display(df[['Action_DQN','OptNextAction','NextHolding_DQN','OptNextHolding','NetPNL_DQN','OptNetPNL', 
                'Risk_DQN', 'OptRisk', 'Reward_DQN', 'OptReward' ]].round(0).describe())

if p['executeRL']: 
    display(df[['Action_Q','OptNextAction','NextHolding_Q','OptNextHolding','NetPNL_Q','OptNetPNL', 
                'Risk_Q', 'OptRisk', 'Reward_Q', 'OptReward' ]].round(0).describe())

# Actions

See histograms for actions made by the algorithms 

In [ ]:
def prime_factors(n):
    i = 2
    factors = []
    while i * i <= n:
        if n % i:
            i += 1
        else:
            n //= i
            factors.append(i)
    if n > 1:
        factors.append(n)
    return factors

def plot_actions_histograms(p,df,actiontag,start,kind=None):

    plt.rcParams['font.size'] = 10
    step = p_post['step_hist_plot']
    counter = 0
    bins = len(np.arange(-p['KLM'][0], p['KLM'][0] + 1, p['KLM'][1]))

    lenght = len(range((p['N_train']//step)))
    primes = prime_factors(lenght)

    if len(primes) == 1:
        m,n = primes[0],1
    elif len(primes) == 2:
        m,n = primes
    elif len(primes) == 3:
        m = primes[0] * primes[1]
        n = primes[2]

    fig = plt.figure(0, figsize=(25,18))
    fig.tight_layout()
    plt.suptitle('{} histograms by steps of {} iterations'.format(actiontag,step), fontsize= 20)
    fig.tight_layout()
    for i in range(m):
        for j in range(n):
            #plt.title('It from {} to {}'. format(start,step*(counter+1)))
            ax = plt.subplot2grid((m,n), (i,j))
            if kind == 'cum':
                ax.set_title('IncW from {} to {}'. format(0,step*(counter+1)))
                ax.hist(df.loc[: step*(counter+1),actiontag],bins=bins)    
            else:
                ax.set_title('RollW from {} to {}'. format(start,step*(counter+1)))
                ax.hist(df.loc[start: step*(counter+1),actiontag],bins=bins)
            start = step*(counter+1)
            counter += 1
    plt.show()
    
def plot_actions_countplot(p,df,actiontag,start,kind=None):

    plt.rcParams['font.size'] = 10
    step = p_post['step_hist_plot']
    counter = 0
    bins = len(np.arange(-p['KLM'][0], p['KLM'][0] + 1, p['KLM'][1]))

    lenght = len(range((p['N_train']//step)))
    primes = prime_factors(lenght)

    if len(primes) == 1:
        m,n = primes[0],1
    elif len(primes) == 2:
        m,n = primes
    elif len(primes) == 3:
        m = primes[0] * primes[1]
        n = primes[2]


    fig = plt.figure(0, figsize=(25,18))
    fig.tight_layout()
    plt.suptitle('{} histograms by steps of {} iterations'.format(actiontag,step), fontsize= 20)
    fig.tight_layout()
    for i in range(m):
        for j in range(n):
            #plt.title('It from {} to {}'. format(start,step*(counter+1)))
            ax = plt.subplot2grid((m,n), (i,j))
            if kind == 'cum':
                ax.set_title('IncW from {} to {}'. format(0,step*(counter+1)))
                #ax.hist(df.loc[: step*(counter+1),actiontag],bins=bins)  
                sns.countplot(x=actiontag, data=df.loc[: step*(counter+1)], palette=sns.color_palette("PRGn", n_colors=bins))
            else:
                ax.set_title('RollW from {} to {}'. format(start,step*(counter+1)))
                sns.countplot(x=actiontag, data=df.loc[start: step*(counter+1)], palette=sns.color_palette("PRGn", n_colors=bins))
            start = step*(counter+1)
            counter += 1
    plt.show()

In [ ]:
p['max_experiences'], p['min_experiences'], p['eps_decay']

In [ ]:
print('Rolling windows')
if p['executeDRL']:
    plot_actions_histograms(p,df,'Action_DQN',p_post['start_hist_plot'])
if p['executeRL']:
    plot_actions_histograms(p,df,'Action_Q',p_post['start_hist_plot'])
if p['executeGP']:
    plot_actions_histograms(p,df,'OptNextAction',p_post['start_hist_plot'])
print()
print('Increasing windows')
if p['executeDRL']:
    plot_actions_histograms(p,df,'Action_DQN',p_post['start_hist_plot'],'cum')
if p['executeRL']:
    plot_actions_histograms(p,df,'Action_Q',p_post['start_hist_plot'],'cum')
if p['executeGP']:
    plot_actions_histograms(p,df,'OptNextAction',p_post['start_hist_plot'],'cum')

In [ ]:
# print('Rolling windows')
# if p['executeDRL']:
#     plot_actions_countplot(p,df,'Action_DQN',p_post['start_hist_plot'])
# if p['executeRL']:
#     plot_actions_countplot(p,df,'Action_Q',p_post['start_hist_plot'])
# print()
# print('Increasing windows')
# if p['executeDRL']:
#     plot_actions_countplot(p,df,'Action_DQN',p_post['start_hist_plot'],'cum')
# if p['executeRL']:
#     plot_actions_countplot(p,df,'Action_Q',p_post['start_hist_plot'],'cum')

# Correlations
See some correlation between action/portfolio of the algorithms with respect to the optimal solution

In [ ]:
def Compute_Corr(df, pairs, start, step, end, tag=None):

    if tag:
        pairs = [[p[0]+'_{}'.format(tag),p[1]] for p in pairs]
    
    corr = pd.DataFrame(columns = ['_'.join(pair) for pair in pairs],
                    index = np.arange(start,
                                      end + 1,
                                      step), dtype=np.float64).drop(0)
    
    for i in corr.index:
        for pair in pairs:     
            corr.at[i,'_'.join(pair)] = np.corrcoef(df[pair[0]].iloc[start:i], 
                                                    df[pair[1]].iloc[start:i])[0][1]    
    return corr

In [ ]:
pairs = [['NextHolding','OptNextHolding'],['Action','OptNextAction'],['Action','returns']]
optpair = [['OptNextAction', 'returns']]
start = 0
step = p_post['step_corr']
end = p['N_train']

if p['executeDRL'] and p['executeGP']:
    corr_DQN = Compute_Corr(df, pairs, start, step, end, 'DQN')
if p['executeRL'] and p['executeGP']:
    corr_Q = Compute_Corr(df, pairs, start, step, end, 'Q')

corr_opt = Compute_Corr(df, optpair, start, step, end)

if p['executeDRL'] and p['executeRL'] and p['executeGP']:
    tot_corr = pd.concat([corr_DQN,corr_Q,corr_opt], axis=1)
elif p['executeDRL'] and not p['executeRL'] and p['executeGP']:
    tot_corr = pd.concat([corr_DQN,corr_opt], axis=1)
elif not p['executeDRL'] and p['executeRL'] and p['executeGP']:
    tot_corr = pd.concat([corr_Q,corr_opt], axis=1)
    
display(tot_corr)

# PnL and Reward

In [ ]:
def ComputeSortinoRatio(series):
    
    mean = np.array(series).mean()
    std = np.array(series[series<0]).std()
    sortino_ratio = (mean/std) * (252 ** 0.5)
    
    return sortino_ratio

def ComputeSharpeRatio(series):
    
    mean = np.array(series).mean()
    std = np.array(series).std()
    sr = (mean/std) * (252 ** 0.5)
    
    return mean, std, sr

In [ ]:
def plot_portfolio_quantities(df, pairs, tag, kind, opt, p):
    
    plt.rcParams['font.size'] = 14
    
    if kind == 'stock':
        if tag:
            pairs = [[p[0]+'_{}'.format(tag),p[1]] for p in pairs]
        # plot quantities as they are
        fig = plt.figure(0, figsize=(25,12))
        fig.tight_layout()
        plt.suptitle('Exposure Quantities for {}'.format(tag), fontsize= 20)
        m,n = 3,1
        counter = 0
        for i in range(m):
            for j in range(n):
                pair = pairs[counter]
                ax = plt.subplot2grid((m,n), (i,j))
                ax.set_title(pair)
                if 'Cost' in pair[0]:
                    ax.hist(df.loc[p['min_experiences']:,pair[0]], bins=5)
                    if opt:
                        ax.hist(df.loc[p['min_experiences']:,pair[1]],alpha=0.5, bins=5)
                        plt.legend([pair[0],pair[1]])
                    else:
                        plt.legend([pair[0]])
                else:
                    ax.plot(df.loc[p['min_experiences']:,pair[0]])
                    if opt:
                        ax.plot(df.loc[p['min_experiences']:,pair[1]],alpha=0.5)
                        plt.legend([pair[0],pair[1]])
                    else:
                        plt.legend([pair[0]])
                counter +=1
        plt.show()

    elif kind == 'cum':
        if tag:
            pairs = [[p[0]+'_{}'.format(tag),p[1]] for p in pairs]
        # plot cumulative quantities
        fig = plt.figure(0,figsize=(25,12))
        fig.tight_layout()
        plt.suptitle('Cumulative Quantities for {}'.format(tag), fontsize= 20)
        m,n = 2,2
        counter = 0
        for i in range(m):
            for j in range(n):
                pair = pairs[counter]
                PNLmean, PNLstd, PNLsr = ComputeSharpeRatio(df[pair[0]])
                if opt and counter < 2:
                    OptPNLmean, OptPNLstd, OptGrossPNLsr = ComputeSharpeRatio(df[pair[1]])
                    pnl_text = AnchoredText(' {} Sharpe Ratio: '.format(pair[0]) + str(np.around(PNLsr,2)) + 
                                                  '\n {} mean: '.format(pair[0]) + str(np.around(PNLmean,2)) + 
                                                  '\n {} PnL std: '.format(pair[0]) + str(np.around(PNLstd,2)) +
                                                  '\n {} Sharpe Ratio: '.format(pair[1]) + str(np.around(OptGrossPNLsr,2)) + 
                                                  '\n {} PnL mean: '.format(pair[1]) + str(np.around(OptPNLmean,2)) + 
                                                  '\n {} PnL std: '.format(pair[1]) + str(np.around(OptPNLstd,2)), 
                                                  loc=4, prop=dict(size=10)) # pad=0, borderpad=0, frameon=False 
                else:
                    pnl_text = AnchoredText(' {} Sharpe Ratio: '.format(pair[0]) + str(np.around(PNLsr,2)) + 
                              '\n {} mean: '.format(pair[0]) + str(np.around(PNLmean,2)) + 
                              '\n {} PnL std: '.format(pair[0]) + str(np.around(PNLstd,2)), 
                              loc=4, prop=dict(size=10)) # pad=0, borderpad=0, frameon=False 
                
                
                ax = plt.subplot2grid((m,n), (i,j))
                ax.set_title(pair)
                ax.plot(df.loc[p['min_experiences']:,pair[0]].cumsum())
                if opt:
                    ax.plot(df.loc[p['min_experiences']:,pair[1]].cumsum(),alpha=0.5)
                    ax.legend([pair[0],pair[1]])
                else:
                    ax.legend([pair[0]])
                if counter < 2:
                    ax.add_artist(pnl_text)                
                counter += 1
        plt.show()

    else:
        print('Choose a proper kind of plot! Either stock or cum!')
        
def plot_grid_equity_curves(p,df,tags, start, kind=None):

    plt.rcParams['font.size'] = 10
    step = p_post['step_eq_plot']
    counter = 0

    lenght = len(range((p['N_train']//step)))
    primes = prime_factors(lenght)

    if len(primes) == 1:
        m,n = primes[0],1
    elif len(primes) == 2:
        m,n = primes
    elif len(primes) == 3:
        m = primes[0] * primes[1]
        n = primes[2]


    fig = plt.figure(0, figsize=(25,18))
    fig.tight_layout()
    plt.suptitle('{} equity curves by steps of {} iterations'.format(tags,step), fontsize= 20)
    fig.tight_layout()
    for i in range(m):
        for j in range(n):
            #plt.title('It from {} to {}'. format(start,step*(counter+1)))
            if kind=='cum':
                PNLmean, PNLstd, PNLsr = ComputeSharpeRatio(df.loc[: step*(counter+1),tags[0]])
                PNLmean2, PNLstd2, PNLsr2 = ComputeSharpeRatio(df.loc[: step*(counter+1),tags[1]])
            else:
                PNLmean, PNLstd, PNLsr = ComputeSharpeRatio(df.loc[start: step*(counter+1),tags[0]])
                PNLmean2, PNLstd2, PNLsr2 = ComputeSharpeRatio(df.loc[start: step*(counter+1),tags[1]])
            
            pnl_text = AnchoredText(' {} Sharpe Ratio: '.format(tags[0]) + str(np.around(PNLsr,2)) + 
                              '\n {} mean: '.format(tags[0]) + str(np.around(PNLmean,2)) + 
                              '\n {} PnL std: '.format(tags[0]) + str(np.around(PNLstd,2)) +
                              '\n {} Sharpe Ratio: '.format(tags[1]) + str(np.around(PNLsr2,2)) + 
                              '\n {} PnL mean: '.format(tags[1]) + str(np.around(PNLmean2,2)) + 
                              '\n {} PnL std: '.format(tags[1]) + str(np.around(PNLstd2,2)), 
                              loc=1, prop=dict(size=10))
            
            ax = plt.subplot2grid((m,n), (i,j))
            if kind == 'cum':
                ax.set_title('IncW from {} to {}'. format(0,step*(counter+1)))
                ax.plot(df.loc[: step*(counter+1),tags].cumsum())
            else:
                ax.set_title('RollW from {} to {}'. format(start,step*(counter+1)))
                ax.plot(df.loc[start: step*(counter+1),tags].cumsum())
            ax.add_artist(pnl_text)  
            ax.legend(tags, loc=4)
            start = step*(counter+1)
            counter += 1
    plt.show()

In [ ]:
stock_pairs = [['NextHolding','OptNextHolding'],['Risk', 'OptRisk'], ['Cost', 'OptCost']]
cum_pairs = [['GrossPNL','OptGrossPNL'],['NetPNL','OptNetPNL'], ['Reward', 'OptReward'], ['Cost', 'OptCost']]

## Exposure quantities

In [ ]:
if p['executeDRL']:
    plot_portfolio_quantities(df,stock_pairs, 'DQN', 'stock', False, p)
    plot_portfolio_quantities(df,stock_pairs, 'DQN', 'stock', True, p)
if p['executeRL']:
    plot_portfolio_quantities(df,stock_pairs, 'Q', 'stock', False, p)
    plot_portfolio_quantities(df,stock_pairs, 'Q', 'stock', True, p)

## Cumulative Portfolio Quantities

In [ ]:
if p['executeDRL']:
    plot_portfolio_quantities(df,cum_pairs, 'DQN', 'cum', False, p)
    plot_portfolio_quantities(df,cum_pairs, 'DQN', 'cum', True, p)
if p['executeRL']:
    plot_portfolio_quantities(df,cum_pairs, 'Q', 'cum', False,p)
    plot_portfolio_quantities(df,cum_pairs, 'Q', 'cum', True,p)

In [ ]:
stock_pairs = [['NextHolding','NextHolding','OptNextHolding'],['Risk','Risk', 'OptRisk'], ['Cost','Cost', 'OptCost']]
cum_pairs = [['GrossPNL','GrossPNL','OptGrossPNL'],['NetPNL','NetPNL','OptNetPNL'], ['Reward','Reward', 'OptReward'], ['Cost','Cost', 'OptCost']]

In [ ]:
def plot_all(df,pairs, tags, kind, opt, p):
    
    plt.rcParams['font.size'] = 14
    
    ps = [[p[0]+'_{}'.format(tags[0]),p[1]+'_{}'.format(tags[1]),p[2]] for p in pairs]
    
    if kind == 'stock':
        # plot quantities as they are
        fig = plt.figure(0, figsize=(25,12))
        fig.tight_layout()
        plt.suptitle('Exposure Quantities for {}'.format(tags), fontsize= 20)
        m,n = 3,1
        counter = 0
        for i in range(m):
            pair = ps[counter]
            
            ax = plt.subplot2grid((m,1), (i,0))
            ax.set_title(pair)
            ax.plot(df.loc[p['min_experiences']:,pair[0]])
            ax.plot(df.loc[p['min_experiences']:,pair[1]])
            if opt:
                ax.plot(df.loc[p['min_experiences']:,pair[2]],alpha=0.5)
                plt.legend([pair[0],pair[1], pair[2]])
            plt.legend([pair[0],pair[1]])
            counter += 1
        plt.show()

    elif kind == 'cum':

        fig = plt.figure(0,figsize=(25,12))
        fig.tight_layout()
        plt.suptitle('Cumulative Quantities for {}'.format(tags), fontsize= 20)
        m,n = 2,2
        counter = 0
        for i in range(m):
            pair = ps[counter]
            for j in range(n):
                pair = ps[counter]
                PNLmean, PNLstd, PNLsr = ComputeSharpeRatio(df[pair[0]])
                PNLmean2, PNLstd2, PNLsr2 = ComputeSharpeRatio(df[pair[1]])
                OptPNLmean, OptPNLstd, OptGrossPNLsr = ComputeSharpeRatio(df[pair[2]])
                if opt and counter < 2:
                    pnl_text = AnchoredText(' {} Sharpe Ratio: '.format(pair[0]) + str(np.around(PNLsr,2)) + 
                                                  '\n {} mean: '.format(pair[0]) + str(np.around(PNLmean,2)) + 
                                                  '\n {} PnL std: '.format(pair[0]) + str(np.around(PNLstd,2)) +
                                            ' \n {} Sharpe Ratio: '.format(pair[1]) + str(np.around(PNLsr2,2)) + 
                                                  '\n {} mean: '.format(pair[1]) + str(np.around(PNLmean2,2)) + 
                                                  '\n {} PnL std: '.format(pair[1]) + str(np.around(PNLstd2,2)) +
                                                  '\n {} Sharpe Ratio: '.format(pair[2]) + str(np.around(OptGrossPNLsr,2)) + 
                                                  '\n {} PnL mean: '.format(pair[2]) + str(np.around(OptPNLmean,2)) + 
                                                  '\n {} PnL std: '.format(pair[2]) + str(np.around(OptPNLstd,2)), 
                                                  loc=4, prop=dict(size=10)) # pad=0, borderpad=0, frameon=False 
                else:
                    pnl_text = AnchoredText(' {} Sharpe Ratio: '.format(pair[0]) + str(np.around(PNLsr,2)) + 
                                              '\n {} mean: '.format(pair[0]) + str(np.around(PNLmean,2)) + 
                                              '\n {} PnL std: '.format(pair[0]) + str(np.around(PNLstd,2))+ 
                                                '\n {} Sharpe Ratio: '.format(pair[1]) + str(np.around(PNLsr2,2)) + 
                                                  '\n {} mean: '.format(pair[1]) + str(np.around(PNLmean2,2)) + 
                                                  '\n {} PnL std: '.format(pair[1]) + str(np.around(PNLstd2,2)), 
                                              loc=4, prop=dict(size=10)) # pad=0, borderpad=0, frameon=False 
                
                
                ax = plt.subplot2grid((m,n), (i,j))
                ax.set_title(pair)
                ax.plot(df.loc[p['min_experiences']:,pair[0]].cumsum(), label=pair[0])
                ax.plot(df.loc[p['min_experiences']:,pair[1]].cumsum(), label=pair[1])
                if opt:
                    ax.plot(df.loc[p['min_experiences']:,pair[2]].cumsum(),alpha=0.5,label=pair[2])
                plt.legend()
                if counter < 2:
                    ax.add_artist(pnl_text)                
                counter += 1
        #plt.show()

    else:
        print('Choose a proper kind of plot! Either stock or cum!')

Picture with the algorithms alltogether (if any)

In [ ]:
# if p['executeDRL'] and p['executeRL']:
#     plot_all(stock_pairs, ['DQN','Q'], 'stock', False, p)
#     plot_all(cum_pairs, ['DQN','Q'], 'cum', False, p)

In [ ]:
if p['executeDRL'] and p['executeRL'] and p['executeGP']:
    plot_all(df, stock_pairs, ['DQN','Q'], 'stock', True, p)
    plot_all(df, cum_pairs, ['DQN','Q'], 'cum', True, p)

## Equity Curves step by step

In [ ]:
if p['executeDRL']:
    print('INCREASING WINDOW')
    plot_grid_equity_curves(p,df,['GrossPNL_DQN','NetPNL_DQN'],p_post['start_hist_plot'],'cum')
    print('ROLLING WINDOW')
    plot_grid_equity_curves(p,df,['GrossPNL_DQN','NetPNL_DQN'],p_post['start_hist_plot'])

In [ ]:
if p['executeRL']:
    print('INCREASING WINDOW')
    plot_grid_equity_curves(p,df,['GrossPNL_Q','NetPNL_Q'],p_post['start_hist_plot'], 'cum')
    print('ROLLING WINDOW')
    plot_grid_equity_curves(p,df,['GrossPNL_Q','NetPNL_Q'],p_post['start_hist_plot'])

# Ex post regressions

In [ ]:
import statsmodels.api as sm

In [ ]:
def opt_trading_rate_disc_loads(p):
    
    f_speed = np.around(np.log(2)/p['HalfLife'],4)
    
    # 1 percent annualized discount rate (same rate of Ritter)
    rho = 1 - np.exp(- p['discount_rate']/260)  

    # kappa is the risk aversion, CostMultiplier the parameter for trading cost
    num1 = (p['kappa'] * ( 1 - rho) + p['CostMultiplier'] *rho)
    num2 = np.sqrt(num1**2 + 4 * p['kappa'] * p['CostMultiplier'] * (1 - rho)**2)
    den = 2* (1 - rho)
    a = (-num1 + num2)/ den

    OptRate = a / p['CostMultiplier']
        
    DiscFactorLoads = p['f_param'] / (1 + f_speed * ((OptRate * p['CostMultiplier']) /  p['kappa']))
    
    return OptRate, DiscFactorLoads, f_speed

def run_optimal_regression(df, p, columns=['OptNextHolding','OptNextAction']):
    
    factors = df[df.columns[df.columns.str.startswith('factor', na=False) == True]].copy()
    regressors = df[columns].copy()
    dfc = pd.concat([factors, regressors], axis=1)
    
    dfc['OptCurrholding'] = dfc[columns[0]] - dfc[columns[1]]
    dfc.drop(dfc.head(1).index, inplace=True)
    dfc.drop(dfc.tail(1).index, inplace=True)
    
    OptRate, DiscFactorLoads, lambdas = opt_trading_rate_disc_loads(p)
    DiscFactorLoads = DiscFactorLoads.reshape(-1,1)
    factors = dfc[df.columns[df.columns.str.startswith('factor', na=False) == True]]
    if len(DiscFactorLoads) == 1:
        retprod = factors * DiscFactorLoads
    else:
        retprod = factors @ DiscFactorLoads
        
    dfc['aim'] = (1/(p['kappa'] * (p['sigma'])**2)) *  retprod
    
    
    X = dfc[['OptCurrholding','aim']]
    y = dfc['OptNextHolding']
    X = sm.add_constant(X.values)
    
    ols_model = sm.OLS(endog=y,exog=X)
    ols_res = ols_model.fit()
    
    return ols_res, (1-OptRate), OptRate

def run_regression(df, p, tag, columns=['CurrHolding','NextHolding']):
    
    factors = df[df.columns[df.columns.str.startswith('factor', na=False) == True]].copy()
    columns = [c+'_{}'.format(tag) for c in columns]
    regressors = df[columns].copy()
    dfc = pd.concat([factors, regressors], axis=1)

    dfc.drop(dfc.head(1).index, inplace=True)
    dfc.drop(dfc.tail(1).index, inplace=True)
    

    OptRate, DiscFactorLoads, lambdas = opt_trading_rate_disc_loads(p)
    DiscFactorLoads = DiscFactorLoads.reshape(-1,1)
    factors = dfc[df.columns[df.columns.str.startswith('factor', na=False) == True]]
    if len(DiscFactorLoads) == 1:
        retprod = factors * DiscFactorLoads
    else:
        retprod = factors @ DiscFactorLoads
        
    dfc['aim'] = (1/(p['kappa'] * (p['sigma'])**2)) *  retprod
#     dfc['aim'] = (1/(p['kappa'] * (p['sigma'])**2)) *  (DiscFactorLoads * \
#                                                         dfc[df.columns[df.columns.str.startswith('factor', na=False) == True]])
    
    X = dfc[['CurrHolding_{}'.format(tag),'aim']]
    y = dfc['NextHolding_{}'.format(tag)]
    #X = sm.add_constant(X.values)
    
    ols_model = sm.OLS(endog=y,exog=X)
    ols_res = ols_model.fit()
    
    return ols_res

In [ ]:
if p['executeDRL']:
    ols_res_DQN = run_regression(df,p,'DQN')
if p['executeRL']:
    ols_res_Q = run_regression(df,p, 'Q')
if p['executeGP']:
    opt_ols_res, c1, c2 = run_optimal_regression(df,p)

## Optimal regression 

In [ ]:
opt_ols_res.summary()

The two coefficient of the optimal regression are: $$x_{t} = \left( 1 - \frac{a}{\lambda}\right) x_{t-1} + \frac{a}{\lambda}aim_{t}$$

In [ ]:
print('Coefficient 1: ',c1)
print('Coefficient 2: ',c2)

while the values obtained by the parameter of the experiment are 

In [ ]:
OptRate, _, _ = opt_trading_rate_disc_loads(p)
print('Value 1: ',c1)
print('Value 2: ',c2)

In [ ]:
# plt.rcParams['figure.figsize'] = (20.0, 10.0)
# sns.distplot(opt_ols_res.resid)

In [ ]:
# sm.qqplot(opt_ols_res.resid, line='r') 
# pass

## DQN regression

In [ ]:
if p['executeDRL']:
    display(ols_res_DQN.summary())

In [ ]:
# if p['executeDRL']:
#     sns.distplot(ols_res_DQN.resid)

In [ ]:
# if p['executeDRL']:
#     sm.qqplot(ols_res_DQN.resid, line='r')
#     pass

## Tabular Q regression

In [ ]:
if p['executeRL']:
    display(ols_res_Q.summary())

In [ ]:
if p['executeRL']:
    ols_res_Q.resid.sum()

In [ ]:
# if p['executeRL']:
#     sns.distplot(ols_res_Q.resid)

In [ ]:
# if p['executeRL']:
#     sm.qqplot(ols_res_Q.resid,line='r')
#     pass

# Learned Value Function

In [ ]:
from utils.DeepLearning import DeepQNetworkModel
from utils.SimulateData import ReturnSampler
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import tensorflow as tf
from matplotlib import cm

In [ ]:
if p['use_GPU']:
    gpu_devices = tf.config.experimental.list_physical_devices('GPU')
    for device in gpu_devices:
        tf.config.experimental.set_memory_growth(device, True)

In [ ]:
def load_DQNmodel(p, data_dir, ckpt=False, ckpt_it=None):
    
    #p['sigma'] = [p['sigma']]
    
    # count the number of inputs as the single volatilities passed as input plus the portfolio holding
    num_inp = 2 # TODO insert number of factors as parameter
    actions = np.arange(-p['KLM'][0], p['KLM'][0] + 1, p['KLM'][1])
    num_actions = len(actions)
    
    model = DeepQNetworkModel(p['seed'], num_inp, p['hidden_units'], num_actions, 
                          p['batch_norm_input'], p['batch_norm_hidden'], p['activation'], p['kernel_initializer'], 
                          p['mom_batch_norm'], p['trainable_batch_norm'],modelname='TrainNet')
    
    if ckpt:
        model.load_weights(os.path.join(data_dir, 'ckpt','DQN_{}_it_weights'.format(ckpt_it)))
    else:
        model.load_weights(os.path.join(data_dir, 'DQN_final_weights'))
    
    return model, actions
    
def plot_learned_DQN(df, model, actions, holding, ax=None, normalize=None, scaler='Standard', less_labels=False, n_less_labels = None, ckpt=False, ckpt_it=None):
    
    sample_Ret = np.linspace(df['returns'].min(), df['returns'].max(), 100)
    if holding == 0:
        holdings = np.zeros(len(sample_Ret))
    else:
        holdings = np.ones(len(sample_Ret)) * holding
    
    states = tf.constant(np.hstack((sample_Ret.reshape(-1,1), holdings.reshape(-1,1))), dtype=tf.float32)
    pred = model(states,training=False)
    
    if normalize:
        if scaler == 'Standard':
            pred = StandardScaler().fit_transform(tf.transpose(pred))
            pred = pred.T
        elif scaler== 'MinMax':
            pred = MinMaxScaler().fit_transform(tf.transpose(pred))
            pred = pred.T
    
    if ax:
        viridis = cm.get_cmap('viridis', pred.shape[1])
        for i in range(pred.shape[1]): 
            if normalize:
                if ckpt:
                    ax.set_title('Learned Q function with {} scaler when h = {} at iteration {}'.format(scaler,int(holdings[0]),ckpt_it))
                else:
                    ax.set_title('Learned Q function with {} scaler when h = {}'.format(scaler,int(holdings[0])))
            else:
                if ckpt:
                    ax.set_title('Learned DQN function when h = {} at iteration {}'.format(int(holdings[0]),ckpt_it))
                else:
                    ax.set_title('Learned DQN function when h = {}'.format(int(holdings[0])))
            if less_labels:
                subset_label_idx = np.round(np.linspace(0, len(actions) - 1, n_less_labels)).astype(int)
                subset_label = actions[subset_label_idx]
                if actions[i] in subset_label:
                    ax.plot(states[:,0],pred[:,i], label=str(actions[i]), c = viridis.colors[i], linewidth=5)
                else:
                    ax.plot(states[:,0],pred[:,i], label='_nolegend_', c = viridis.colors[i], linewidth=5)
            else:
                ax.plot(states[:,0],pred[:,i], label=str(actions[i]), c = viridis.colors[i], linewidth=5)
            ax.set_xlabel('Returns')
            ax.set_ylabel('Q value')
            ax.legend()
        
    else:
        viridis = cm.get_cmap('viridis', pred.shape[1])
        plt.figure()
        for i in range(pred.shape[1]):
            if normalize:
                if ckpt:
                    plt.title('Learned Q function with {} scaler when h = {} at iteration {}'.format(scaler,int(holdings[0]),ckpt_it))
                else:
                    plt.title('Learned Q function with {} scaler when h = {}'.format(scaler,int(holdings[0])))
            else:
                if ckpt:
                    plt.title('Learned DQN function when h = {} at iteration {}'.format(int(holdings[0]),ckpt_it))
                else:
                    plt.title('Learned DQN function when h = {}'.format(int(holdings[0])))
            if less_labels:
                subset_label_idx = np.round(np.linspace(0, len(actions) - 1, n_less_labels)).astype(int)
                subset_label = actions[subset_label_idx]
                if actions[i] in subset_label:
                    plt.plot(states[:,0],pred[:,i], label=str(actions[i]), c = viridis.colors[i], linewidth=5)
                else:
                    plt.plot(states[:,0],pred[:,i], label='_nolegend_', c = viridis.colors[i], linewidth=5)
            else:
                plt.plot(states[:,0],pred[:,i], label=str(actions[i]), c = viridis.colors[i], linewidth=5)
            plt.xlabel('Returns')
            plt.ylabel('Q value')
            plt.legend()

        
def learned_DQN_grid(df, model, actions, holdings, normalize=None):
    
    lenght = len(holdings)
    primes = prime_factors(lenght)
    if len(primes) == 1:
        m,n = primes[0],1
    elif len(primes) == 2:
        m,n = primes
    elif len(primes) == 3:
        m = primes[0] * primes[1]
        n = primes[2]
    counter=0
        
    fig = plt.figure(0, figsize=(25,18))
    fig.tight_layout()
    for i in range(m):
        for j in range(n):
            ax = plt.subplot2grid((m,n), (i,j))
            if normalize:
                plot_learned_DQN(df, model,actions, holdings[counter], ax, normalize, less_labels=True, n_less_labels=6)
            else:
                plot_learned_DQN(df, model,actions, holdings[counter], ax, less_labels=True, n_less_labels=6)
            counter += 1
            
def plot_QValueFunction(QTable,holding=0, aggregation=False, ax=None, normalize=None, scaler='Standard', less_labels=False, n_less_labels = None):

    '''
    This function accepts the QTable and a dict of parameters as arguments to plot the value function 
    with respect to the price
    '''

    # select value for actions
    act = QTable.columns
    # get color from colormaps
    viridis = cm.get_cmap('viridis', len(QTable.columns))
    if aggregation:
        Q_price = QTable.groupby(level = [0]).sum()
        # select values for xaxis
        ret = QTable.index.get_level_values(0).unique()

        fig, ax = plt.subplots(1,1)
        for i in range(0,len(Q_price.columns)):
            ax.scatter(ret,Q_price.iloc[:,i],label=Q_price.columns[i], s=15,
                       c = viridis.colors[i])

    else:
        Q_price = QTable[QTable.index.get_level_values('Holding') == holding]
        Q_price.index = Q_price.index.droplevel(1)
        # index
        idx = np.round(np.linspace(0, len(Q_price.index) - 1, 500)).astype(int)
        Q_price = Q_price.iloc[idx]
        # select values for xaxis
        ret = Q_price.index
        
        if ax:
            if normalize:
                if scaler == 'Standard':
                    Q_price = StandardScaler().fit_transform(Q_price.T)
                    Q_price = Q_price.T
                elif scaler == 'MinMax':
                    Q_price = MinMaxScaler().fit_transform(Q_price.T)
                    Q_price = Q_price.T
                    
                ax.set_title('Learned Q function with {} scaler when h = {}'.format(scaler,int(holding)))
                for i in range(0,Q_price.shape[1]):
                    if less_labels:
                        subset_label_idx = np.round(np.linspace(0, len(act) - 1, n_less_labels)).astype(int)
                        subset_label = act[subset_label_idx]
                        if act[i] in subset_label:
                            ax.scatter(ret,Q_price[:,i],label=act[i], s=15,c = np.repeat(np.atleast_2d(viridis.colors[i]), len(Q_price[:,i]), axis = 0))
                        else:
                            ax.scatter(ret,Q_price[:,i],label='_nolegend_', s=15,c = np.repeat(np.atleast_2d(viridis.colors[i]), len(Q_price[:,i]), axis = 0))
                    else:
                        ax.scatter(ret,Q_price[:,i],label=act[i], s=15, c = np.repeat(np.atleast_2d(viridis.colors[i]), len(Q_price[:,i]), axis = 0))
            else:
                ax.set_title('Learned Q function when h = {}'.format(int(holding)))
                for i in range(0,len(Q_price.columns)):
                    if less_labels:
                        subset_label_idx = np.round(np.linspace(0, len(Q_price.columns) - 1, n_less_labels)).astype(int)
                        subset_label = Q_price.columns[subset_label_idx]
                        if Q_price.columns[i] in subset_label:
                            ax.scatter(ret,Q_price.iloc[:,i].values,label=Q_price.columns[i], s=15,c = np.repeat(np.atleast_2d(viridis.colors[i]),
                                                                                                             len(Q_price.iloc[:,i].values), axis = 0))
                        else:
                            ax.scatter(ret,Q_price.iloc[:,i].values,label='_nolegend_', s=15,c = np.repeat(np.atleast_2d(viridis.colors[i]),
                                                                                                             len(Q_price.iloc[:,i].values), axis = 0))
                    else:
                        ax.scatter(ret,Q_price.iloc[:,i].values,label=Q_price.columns[i], s=15,c = np.repeat(np.atleast_2d(viridis.colors[i]),
                                                                                                             len(Q_price.iloc[:,i].values), axis = 0))


    
            ax.tick_params(axis='both', which='major', labelsize=17)
            ax.tick_params(axis='both', which='minor', labelsize=17)
            plt.xlabel('Returns',fontsize=17)
            plt.ylabel(r'$\hat{Q}((holding,p),a)$',fontsize=17)
            plt.legend(loc=0)
        else:
            
            fig, ax = plt.subplots(1,1)
            if normalize:
                if scaler == 'Standard':
                    Q_price = StandardScaler().fit_transform(Q_price.T)
                    Q_price = Q_price.T
                elif scaler == 'MinMax':
                    Q_price = MinMaxScaler().fit_transform(Q_price.T)
                    Q_price = Q_price.T
                    
                for i in range(0,Q_price.shape[1]):
                    if less_labels:
                        subset_label_idx = np.round(np.linspace(0, len(act) - 1, n_less_labels)).astype(int)
                        subset_label = act[subset_label_idx]
                        if act[i] in subset_label:
                            ax.scatter(ret,Q_price[:,i],label=act[i], s=15,c = np.repeat(np.atleast_2d(viridis.colors[i]), len(Q_price[:,i]), axis = 0))
                        else:
                            ax.scatter(ret,Q_price[:,i],label='_nolegend_', s=15,c = np.repeat(np.atleast_2d(viridis.colors[i]), len(Q_price[:,i]), axis = 0))
                    else:
                        ax.scatter(ret,Q_price[:,i],label=act[i], s=15, c = np.repeat(np.atleast_2d(viridis.colors[i]), len(Q_price[:,i]), axis = 0))
                    ax.set_title('Learned Q function with {} scaler when h = {}'.format(scaler,int(holding)))
            else:
                for i in range(0,len(Q_price.columns)):
                    if less_labels:
                        subset_label_idx = np.round(np.linspace(0, len(Q_price.columns) - 1, n_less_labels)).astype(int)
                        subset_label = Q_price.columns[subset_label_idx]
                        if Q_price.columns[i] in subset_label:
                            ax.scatter(ret,Q_price.iloc[:,i].values,label=Q_price.columns[i], s=15,
                               c = np.repeat(np.atleast_2d(viridis.colors[i]), len(Q_price.iloc[:,i].values), axis = 0))
                        else:
                            ax.scatter(ret,Q_price.iloc[:,i].values,label='_nolabel_', s=15,
                               c = np.repeat(np.atleast_2d(viridis.colors[i]), len(Q_price.iloc[:,i].values), axis = 0))
                    else:
                        ax.scatter(ret,Q_price.iloc[:,i].values,label=Q_price.columns[i], s=15,
                               c = np.repeat(np.atleast_2d(viridis.colors[i]), len(Q_price.iloc[:,i].values), axis = 0))
                    ax.set_title('Learned Q function with {} scaler when h = {}'.format(scaler,int(holding)))

                    ax.set_title('Learned Q function when h = {}'.format(int(holding)))

    
            ax.tick_params(axis='both', which='major', labelsize=17)
            ax.tick_params(axis='both', which='minor', labelsize=17)
            plt.xlabel('Returns',fontsize=17)
            plt.ylabel(r'$\hat{Q}((holding,p),a)$',fontsize=17)
            plt.legend(loc=0)
    return Q_price

def learned_Q_grid(Q, holdings, normalize=None):
    
    lenght = len(holdings)
    primes = prime_factors(lenght)
    if len(primes) == 1:
        m,n = primes[0],1
    elif len(primes) == 2:
        m,n = primes
    elif len(primes) == 3:
        m = primes[0] * primes[1]
        n = primes[2]
    counter=0
        
    fig = plt.figure(0, figsize=(25,18))
    fig.tight_layout()
    for i in range(m):
        for j in range(n):
            ax = plt.subplot2grid((m,n), (i,j))
            if normalize:
                plot_QValueFunction(Q,holdings[counter], ax=ax, normalize=normalize, less_labels=True, n_less_labels=6)
            else:
                plot_QValueFunction(Q,holdings[counter], ax=ax, less_labels=True, n_less_labels=6)
            counter += 1  
            
            

    

In [ ]:
model, actions = load_DQNmodel(p, data_dir)
plot_learned_DQN(df, model, actions, 0, less_labels=True, n_less_labels=6)

In [ ]:
if p['executeRL']:
    plt.rcParams['figure.figsize'] = (20.0, 10.0)
    Q_price = plot_QValueFunction(Q,0, less_labels=True, n_less_labels=6)

In [ ]:
if not p['std_rwds']:
    if p['executeDRL'] and p['executeRL']:
        m,n = 2,3
        normalize = True
        fig = plt.figure(0, figsize=(40,25))
        fig.tight_layout()
        for i in range(m):
            for j in range(n):
                ax = plt.subplot2grid((m,n), (i,j))
                if i == 0 and j == 0:
                    plot_learned_DQN(df, model, actions, 0, ax=ax, less_labels=True, n_less_labels=6)
                elif i == 0 and j == 1:
                    plot_learned_DQN(df, model, actions, 0, ax=ax, normalize=normalize, scaler='Standard', less_labels=True, n_less_labels=6)
                elif i == 0 and j == 2:
                    plot_learned_DQN(df, model, actions, 0, ax=ax, normalize=normalize, scaler='MinMax', less_labels=True, n_less_labels=6)
                elif i == 1 and j == 0:
                    plot_QValueFunction(Q,0, ax=ax, less_labels=True, n_less_labels=6)
                elif i == 1 and j == 1:
                    plot_QValueFunction(Q,0, ax=ax, normalize=normalize, scaler='Standard', less_labels=True, n_less_labels=6)
                elif i == 1 and j == 2:
                    plot_QValueFunction(Q,0, ax=ax, normalize=normalize, scaler='MinMax', less_labels=True, n_less_labels=6)
                else:
                    print('Out of range!')

## DQN Checkpoints

In [ ]:
def plot_DQN_checkpoints(data_dir):
    
    checkpoints = [each for each in os.listdir(os.path.join(data_dir,'ckpt')) if each.endswith('.index')]
    its = [re.findall(r'\d+', ckpts) for ckpts in checkpoints]
    its = sorted([int(item) for sublist in its for item in sublist])
    
    for it in its:
        model, actions = load_DQNmodel(p, data_dir, ckpt=True, ckpt_it=it)
        plot_learned_DQN(df, model, actions, 0, less_labels=True, n_less_labels=6, ckpt=True, ckpt_it=it)
        plt.show()

In [ ]:
plot_DQN_checkpoints(data_dir)

In [ ]:
##### RANDOM HOLDINGS
# np.random.choice(df['NextHolding_DQN'].value_counts().index.tolist(),4)
# learned_DQN_holding = np.random.choice(df['NextHolding_DQN'].value_counts().index.tolist(),5)
# learned_DQN_holding = [i for i in learned_DQN_holding if i != 0][:4]
# learned_DQN_grid(df, model, actions, learned_DQN_holding)
# plt.show()
# if not p['std_rwds']:
#     learned_DQN_grid(df, model, actions, learned_DQN_holding, True)
#     plt.show()

# if p['executeRL']:
#     learned_Q_holding = np.random.choice(df['NextHolding_Q'].value_counts().index.tolist(),5)
#     learned_Q_holding = [i for i in learned_Q_holding if i != 0][:4]
#     learned_Q_grid(Q, learned_Q_holding)
#     plt.show()
# if p['executeRL'] and not p['std_rwds']:
#     plt.rcParams['figure.figsize'] = (20.0, 10.0)
#     learned_Q_grid(Q, learned_Q_holding, normalize=True)
#     plt.show()

In [ ]:
#### CODE TO TEST PLOTS
# # select value for actions
# act = Q.columns
# # get color from colormaps
# viridis = cm.get_cmap('viridis', len(Q.columns))
# holding=0

# Q_price = Q[Q.index.get_level_values('Holding') == holding]
# Q_price.index = Q_price.index.droplevel(1)
# # index
# idx = np.round(np.linspace(0, len(Q_price.index) - 1, 500)).astype(int)
# Q_price = Q_price.iloc[idx]
# # select values for xaxis
# ret = Q_price.index

# fig, ax = plt.subplots(1,1)
# #Q_price_std = MinMaxScaler().fit_transform(Q_price)
# Q_price_orig = Q_price.values

# for i in range(0,Q_price.shape[1]):
#     if i==0 or i == 4:
#         ax.scatter(ret,Q_price_orig[:,i],label=act[i], s=15, c = np.repeat(np.atleast_2d(viridis.colors[i]), len(Q_price_orig[:,i]), axis = 0))
#     plt.legend()
    
# fig, ax = plt.subplots(1,1)
# Q_price_std = MinMaxScaler().fit_transform(Q_price.T)
# Q_price_std = Q_price_std.T

# for i in range(0,Q_price.shape[1]):

#     ax.scatter(ret,Q_price_std[:,i],label=act[i], s=15, c = np.repeat(np.atleast_2d(viridis.colors[i]), len(Q_price_std[:,i]), axis = 0))
#     plt.legend()

# Best Actions

In [ ]:
def plot_BestQActions(QTable,aggregation=False, holding=0, ax=None, optsol=None):

    if aggregation:
        Q_price = QTable.groupby(level = [0]).sum()
        Q_action = Q_price.idxmax(axis = 1).astype(float)

        fig, ax = plt.subplots(1,1)
        ax.plot(Q_action)

    else:
        Q_price = QTable[QTable.index.get_level_values('Holding') == holding]
        Q_price.index = Q_price.index.droplevel(1)
        Q_action = Q_price.idxmax(axis = 1).astype(float)

        if ax:
            ax.plot(Q_action,label = 'Q Action')
            if optsol:
                ret = np.array(Q_price.index.unique())
                optactions = compute_optimal_actions(ret,p,holding)
                ax.plot(Q_price.index.unique(),optactions/10, label='Optimal Action')
                plt.legend()
        else:
            fig, ax = plt.subplots(1,1)
            ax.plot(Q_action, label = 'Q Action')
            if optsol:
                ret = np.array(Q_price.index.unique())
                optactions = compute_optimal_actions(ret,p,holding)
                ax.plot(Q_price.index.unique(),optactions/10, label='Optimal Action')
                plt.legend()
    plt.title('Best Q action by level of returns when holding={}'.format(int(holding)))
    ax.tick_params(axis='both', which='major', labelsize=17)
    ax.tick_params(axis='both', which='minor', labelsize=17)
    plt.xlabel('Returns',fontsize=17)
    plt.ylabel('Best Action',fontsize=17)

def BestQActions_grid(Q, holdings, optsol=None):
    
    lenght = len(holdings)
    primes = prime_factors(lenght)
    if len(primes) == 1:
        m,n = primes[0],1
    elif len(primes) == 2:
        m,n = primes
    elif len(primes) == 3:
        m = primes[0] * primes[1]
        n = primes[2]
    counter=0
        
    fig = plt.figure(0, figsize=(25,18))
    fig.tight_layout()
    for i in range(m):
        for j in range(n):
            ax = plt.subplot2grid((m,n), (i,j))
            plot_BestQActions(Q,aggregation=False, holding=holdings[counter], ax=ax, optsol=optsol)
            counter += 1    
    
def plot_BestDQNActions(p, df, model, holding, ax=None, optsol=None):
    
    actions = np.arange(-p['KLM'][0], p['KLM'][0] + 1, p['KLM'][1])
    sample_Ret = np.linspace(df['returns'].min(), df['returns'].max(), 100)
    if holding == 0:
        holdings = np.zeros(len(sample_Ret))
    else:
        holdings = np.ones(len(sample_Ret)) * holding
    
    states = tf.constant(np.hstack((sample_Ret.reshape(-1,1), holdings.reshape(-1,1))), dtype=tf.float32)
    pred = model(states,training=False)

    max_action = actions[tf.math.argmax(pred, axis=1)]
    if ax:
        ax.plot(sample_Ret,max_action,linewidth=5, label='DQN Action')
        if optsol:
            optactions = compute_optimal_actions(sample_Ret,p,holding)
            ax.plot(sample_Ret,optactions/10,linewidth=5, label='Optimal Action')
            plt.legend()
        ax.set_title('Best DQN action by level of returns when holding={}'.format(int(holding)))
        ax.set_xlabel('Returns')
        ax.set_ylabel('Best DQN Action')
    else:
        plt.figure()
        plt.title('Best DQN action by level of returns when holding={}'.format(int(holding)))
        plt.plot(sample_Ret,max_action,linewidth=5, label='DQN Actions')
        if optsol:
            optactions = compute_optimal_actions(sample_Ret,p,holding)
            plt.plot(sample_Ret,optactions/10,linewidth=5, label='Optimal Action')
            plt.legend()
        plt.xlabel('Returns')
        plt.ylabel('Best DQN Action')
    
def BestDQNActions_grid(p, df, model, holdings, optsol= None):
    
    lenght = len(holdings)
    primes = prime_factors(lenght)
    if len(primes) == 1:
        m,n = primes[0],1
    elif len(primes) == 2:
        m,n = primes
    elif len(primes) == 3:
        m = primes[0] * primes[1]
        n = primes[2]
    counter=0
        
    fig = plt.figure(0, figsize=(25,18))
    fig.tight_layout()
    for i in range(m):
        for j in range(n):
            ax = plt.subplot2grid((m,n), (i,j))
            plot_BestDQNActions(p, df, model, holdings[counter], ax, optsol)
            counter += 1   
            
def compute_optimal_actions(ret,p, holding):
    
    OptRate, DiscFactorLoads, lambdas = opt_trading_rate_disc_loads(p)
    aim = (1/(p['kappa'] * (p['sigma'])**2)) *  ((DiscFactorLoads/p['f_param']) * ret)
    nexthold = (1-OptRate) * holding + OptRate * aim
    actions = nexthold - holding
    return actions

In [ ]:
# ret = np.array(Q.index.get_level_values(0).unique())
# OptRate, DiscFactorLoads, lambdas = opt_trading_rate_disc_loads(p)
# # aim = (1/(p['kappa'] * (p['sigma'][0])**2)) *  (DiscFactorLoads * df['factor_10'].values)
# aim = (1/(p['kappa'] * (p['sigma'][0])**2)) *  ((DiscFactorLoads/p['f_param']) * ret)
# fig, ax = plt.subplots(1,1)
# holds = [0, 100000, 1000000,10000000, -10000000,-100000, -1000000]
# for h in holds:
#     nexthold = (1-OptRate) * h + OptRate * aim
#     actions = nexthold - h
#     ax.plot(ret,actions)
# plt.legend(holds)

In [ ]:
plot_BestDQNActions(p, df, model, 0)
if df.filter(like='factor').shape[1] == 1:
    plot_BestDQNActions(p, df, model, 0, optsol=True)
# BestDQNActions_grid(p, df, model, learned_DQN_holding)
# if df.filter(like='factor').shape[1] == 1:
#     BestDQNActions_grid(p, df, model, learned_DQN_holding, optsol=True)

In [ ]:
if p['executeRL']:
    plot_BestQActions(Q,aggregation=False, holding=0)
    if df.filter(like='factor').shape[1] == 1:
        plot_BestQActions(Q,aggregation=False, holding=0, optsol=True)
#     BestQActions_grid(Q,learned_Q_holding)
#     if df.filter(like='factor').shape[1] == 1:
#         BestQActions_grid(Q,learned_Q_holding, optsol=True)

# Out-of-sample Test

## Stats

In [ ]:
df_test.head()

In [ ]:
# stats for algorithm solution
if p['executeDRL']: 
    display(df_test.filter(like='_DQN').describe())

TabQ stats (if any)

In [ ]:
# stats for optimal solution
if p['executeRL']:
    display(df_test.filter(like='_Q').describe())

Optimal stats

In [ ]:
if p['executeGP']:
    display(df_test.filter(like='Opt').describe())

Comparison with optimal stats

In [ ]:
# compare the two
if p['executeDRL']: 
    display(df_test[['Action_DQN','OptNextAction','NextHolding_DQN','OptNextHolding','NetPNL_DQN','OptNetPNL', 
                'Risk_DQN', 'OptRisk', 'Reward_DQN', 'OptReward' ]].round(0).describe())

if p['executeRL']: 
    display(df_test[['Action_Q','OptNextAction','NextHolding_Q','OptNextHolding','NetPNL_Q','OptNetPNL', 
                'Risk_Q', 'OptRisk', 'Reward_Q', 'OptReward' ]].round(0).describe())

## Action histograms

In [ ]:
actions = np.arange(-p['KLM'][0], p['KLM'][0] + 1, p['KLM'][1])
num_actions = len(actions)
if p['executeDRL']:
    plt.figure()
    df_test['Action_DQN'].plot(kind='hist', bins=num_actions, label='Action_DQN')
    plt.legend()
if p['executeRL']:
    plt.figure()
    df_test['Action_Q'].plot(kind='hist', bins=num_actions, label='Action_Q')
    plt.legend()
if p['executeGP']:
    plt.figure()
    df_test['OptNextAction'].plot(kind='hist', bins=num_actions, label='OptNextAction')
    plt.legend()

## Correlations

In [ ]:
pairs = [['NextHolding','OptNextHolding'],['Action','OptNextAction'],['Action','returns']]
optpair = [['OptNextAction', 'returns']]
start = 0
step = p['N_test']
end = p['N_test']

if p['executeDRL'] and p['executeGP']:
    corr_DQN = Compute_Corr(df_test, pairs, start, step, end, 'DQN')
if p['executeRL'] and p['executeGP']:
    corr_Q = Compute_Corr(df_test, pairs, start, step, end, 'Q')

corr_opt = Compute_Corr(df_test, optpair, start, step, end)

if p['executeDRL'] and p['executeRL'] and p['executeGP']:
    tot_corr = pd.concat([corr_DQN,corr_Q,corr_opt], axis=1)
elif p['executeDRL'] and not p['executeRL'] and p['executeGP']:
    tot_corr = pd.concat([corr_DQN,corr_opt], axis=1)
elif not p['executeDRL'] and p['executeRL'] and p['executeGP']:
    tot_corr = pd.concat([corr_Q,corr_opt], axis=1)
    
display(tot_corr)

## PnL and Rewards

In [ ]:
stock_pairs = [['NextHolding','OptNextHolding'],['Risk', 'OptRisk'], ['Cost', 'OptCost']]
cum_pairs = [['GrossPNL','OptGrossPNL'],['NetPNL','OptNetPNL'], ['Reward', 'OptReward'], ['Cost', 'OptCost']]

In [ ]:
if p['executeDRL']:
    plot_portfolio_quantities(df_test,stock_pairs, 'DQN', 'stock', False, p)
    plot_portfolio_quantities(df_test,stock_pairs, 'DQN', 'stock', True, p)
if p['executeRL']:
    plot_portfolio_quantities(df_test,stock_pairs, 'Q', 'stock', False, p)
    plot_portfolio_quantities(df_test,stock_pairs, 'Q', 'stock', True, p)

In [ ]:
if p['executeDRL']:
    plot_portfolio_quantities(df_test,cum_pairs, 'DQN', 'cum', False, p)
    plot_portfolio_quantities(df_test,cum_pairs, 'DQN', 'cum', True, p)
if p['executeRL']:
    plot_portfolio_quantities(df_test,cum_pairs, 'Q', 'cum', False,p)
    plot_portfolio_quantities(df_test,cum_pairs, 'Q', 'cum', True,p)

## Out-of.sample Regression

In [ ]:
if p['executeDRL']:
    ols_res_DQN = run_regression(df_test,p,'DQN')
if p['executeRL']:
    ols_res_Q = run_regression(df_test,p, 'Q')
if p['executeGP']:
    opt_ols_res, c1, c2 = run_optimal_regression(df_test,p)

In [ ]:
print('Coefficient 1: ',c1)
print('Coefficient 2: ',c2)

### Optimal regression 

In [ ]:
opt_ols_res.summary()

In [ ]:
OptRate, _, _ = opt_trading_rate_disc_loads(p)
print('Value 1: ',c1)
print('Value 2: ',c2)

In [ ]:
# plt.rcParams['figure.figsize'] = (10.0, 8.0)
# sns.distplot(opt_ols_res.resid)

In [ ]:
# sm.qqplot(opt_ols_res.resid, line='r') 
# pass

### DQN regression

In [ ]:
if p['executeDRL']:
    display(ols_res_DQN.summary())

In [ ]:
# if p['executeDRL']:
#     sns.distplot(ols_res_DQN.resid)

### Tabular Q regression

In [ ]:
if p['executeRL']:
    display(ols_res_Q.summary())

In [ ]:
if p['executeRL']:
    ols_res_Q.resid.sum()

In [ ]:
# if p['executeRL']:
#     sns.distplot(ols_res_Q.resid)

In [ ]:
# trick to disable code before exporting the notebook results as pdf
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')